In [2]:
from planetarypy.pds import get_index, list_indexes, print_pds_tree

In [3]:
list_indexes?

Signature: list_indexes(mission: str, instrument: str) -> List[str]
Docstring:
List all indexes for a given mission and instrument.

Args:
    mission: Mission name (e.g., 'cassini', 'mro')
    instrument: Instrument name (e.g., 'iss', 'ctx')

Returns:
    List of index names

Examples:
    >>> from planetarypy.pds.utils import list_indexes
    >>> indexes = list_indexes('cassini', 'iss')
    >>> print(indexes)
    ['index', 'moon_summary', 'ring_summary', 'saturn_summary']
File:      ~/Dropbox/src/planetarypy/src/planetarypy/pds/utils.py
Type:      function

In [4]:
list_indexes("mro", "ctx")

['edr']

In [5]:
get_index("mro.ctx.edr")

,VOLUME_ID,FILE_SPECIFICATION_NAME,ORIGINAL_PRODUCT_ID,PRODUCT_ID,IMAGE_TIME,INSTRUMENT_ID,INSTRUMENT_MODE_ID,LINE_SAMPLES,LINES,SPATIAL_SUMMING,...,SUB_SOLAR_LATITUDE,SUB_SPACECRAFT_LONGITUDE,SUB_SPACECRAFT_LATITUDE,SOLAR_DISTANCE,SOLAR_LONGITUDE,LOCAL_TIME,IMAGE_SKEW_ANGLE,RATIONALE_DESC,DATA_QUALITY_DESC,ORBIT_NUMBER
0,MROX_0001,DATA/CRU_000001_9999_XN_99N999W.IMG,4A_04_0001000400,CRU_000001_9999_XN_99N999W,2005-08-30 15:40:21.549,CTX,NIFL,5056,1024,1,...,0.0,0.0,0.0,0.0,278.89,10.16,0.0,Instrument checkout image of space ...,OK,-4242
1,MROX_0001,DATA/CRU_000002_9999_XN_99N999W.IMG,4A_04_0001000500,CRU_000002_9999_XN_99N999W,2005-09-08 15:59:45.313,CTX,NIFL,5056,15360,1,...,0.0,0.0,0.0,0.0,284.48,4.6,0.0,Calibration image of the Moon ...,OK,-4126
2,MROX_0001,DATA/CRU_000003_9999_XN_99N999W.IMG,4A_04_0001000600,CRU_000003_9999_XN_99N999W,2005-09-08 16:03:37.927,CTX,NIFL,5056,2048,1,...,0.0,0.0,0.0,0.0,284.48,4.66,0.0,Calibration image of Omega Centauri (globular ...,OK,-4126
3,MROX_0001,DATA/CRU_000004_9999_XN_99N999W.IMG,4A_04_0001000700,CRU_000004_9999_XN_99N999W,2005-09-08 16:08:23.841,CTX,NIFL,5056,2048,1,...,0.0,0.0,0.0,0.0,284.48,4.74,0.0,Calibration image of Omega Centauri (globular ...,OK,-4126
4,MROX_0001,DATA/CRU_000005_9999_XN_99N999W.IMG,4A_04_0001000800,CRU_000005_9999_XN_99N999W,2005-09-08 16:11:18.649,CTX,NIFL,5056,21504,1,...,0.0,0.0,0.0,0.0,284.48,4.79,0.0,Calibration image of the Moon ...,OK,-4126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152882,MROX_5010,DATA/V12_084836_1506_XI_29S071W.IMG,4A_04_11AF027A00,V12_084836_1506_XI_29S071W,2024-08-31 18:21:22.286,CTX,ITL,5056,11264,1,...,-15.57,71.68,-29.55,219041031.9,321.41,14.46,90.1,CTX stereo with K16_059595_1506_XI_29S071W ...,OK,84836
152883,MROX_5010,DATA/V12_084837_0940_XI_86S052W.IMG,4A_04_11AF027B00,V12_084837_0940_XI_86S052W,2024-08-31 19:55:10.188,CTX,ITL,5056,52224,1,...,-15.56,52.97,-86.11,219054877.1,321.45,17.29,90.4,South polar residual cap summer mosaic campaig...,OK,84837
152884,MROX_5010,DATA/V12_084838_0929_XI_87S066W.IMG,4A_04_11AF027C00,V12_084838_0929_XI_87S066W,2024-08-31 21:46:56.246,CTX,ITL,5056,52224,1,...,-15.54,63.4,-86.92,219069205.0,321.49,18.16,90.4,South polar residual cap summer mosaic campaig...,OK,84838
152885,MROX_5010,DATA/V12_084839_0926_XI_87S060W.IMG,4A_04_11AF027D00,V12_084839_0926_XI_87S060W,2024-08-31 23:38:40.207,CTX,ITL,5056,52224,1,...,-15.53,61.31,-87.39,219083559.2,321.54,20.29,90.4,South polar residual cap summer mosaic campaig...,OK,84839


In [6]:
print_pds_tree()

PDS Indexes Configuration:
├── cassini
│   ├── iss
│   │   ├── index
│   │   ├── inventory
│   │   ├── moon_summary
│   │   ├── ring_summary
│   │   └── saturn_summary
│   └── uvis
│       ├── index
│       ├── moon_summary
│       ├── ring_summary
│       ├── saturn_summary
│       ├── supplemental_index
│       └── versions
├── go
│   └── ssi
│       └── edr
├── lro
│   ├── diviner
│   │   ├── edr1
│   │   ├── edr2
│   │   ├── rdr1
│   │   └── rdr2
│   ├── lola
│   │   ├── edr
│   │   └── rdr
│   └── lroc
│       └── edr
└── mro
    ├── crism
    │   └── mtrdr
    ├── ctx
    │   └── edr
    └── hirise
        ├── dtm
        ├── edr
        └── rdr


In [8]:
df = get_index("cassini.iss.index", refresh=False)

COISS_2999_index.lbl:   0%|          | 0/8420 [00:00<?, ?it/s]

COISS_2999_index.tab: 0it [00:00, ?it/s]

Downloaded /Users/maye/planetarypy_data/missions/cassini/iss/index/COISS_2999_index.lbl and /Users/maye/planetarypy_data/missions/cassini/iss/index/COISS_2999_index.tab
Reading index to memory for conversion to parquet. Will take up lots of memory for a bit.


Loading index in chunks:   0%|          | 0/81 [00:00<?, ?it/s]

/Users/maye/Dropbox/src/planetarypy/src/planetarypy/pds/index_labels.py:173: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[column] = pd.to_datetime(df[column])
/Users/maye/Dropbox/src/planetarypy/src/planetarypy/pds/index_labels.py:173: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[column] = pd.to_datetime(df[column])
/Users/maye/Dropbox/src/planetarypy/src/planetarypy/pds/index_labels.py:173: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[column] = pd.to_datetime(df[column])
/Users/maye/Dropbox/src/planetarypy/src/planetarypy/pds/index_labels.py:173: UserWarning: C

Convert time strings to datetime objects.


/Users/maye/miniforge3/envs/py313/lib/python3.13/site-packages/pandas/core/dtypes/cast.py:1056: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
/Users/maye/miniforge3/envs/py313/lib/python3.13/site-packages/pandas/core/dtypes/cast.py:1080: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


Storing into parquet.
Finished. Enjoy your freshly baked PDS Index. :


In [9]:
df.head()

,FILE_NAME,FILE_SPECIFICATION_NAME,VOLUME_ID,ANTIBLOOMING_STATE_FLAG,BIAS_STRIP_MEAN,CALIBRATION_LAMP_STATE_FLAG,COMMAND_FILE_NAME,COMMAND_SEQUENCE_NUMBER,DARK_STRIP_MEAN,DATA_CONVERSION_TYPE,...,TWIST_ANGLE,TARGET_LIST,UPPER_LEFT_LATITUDE,UPPER_LEFT_LONGITUDE,UPPER_RIGHT_LATITUDE,UPPER_RIGHT_LONGITUDE,DATA_SET_NAME,INSTRUMENT_HOST_ID,PRODUCT_TYPE,STANDARD_DATA_PRODUCT_ID
0,N1454725799_1.IMG,data/1454725799_1455008789/N1454725799_1.IMG,COISS_2001,OFF,14.869863,N/A,OPNAV_848_3.ioi ...,8,0.0,12BIT,...,89.513591,"S2_2004,HELENE,TELESTO,RHEA ...",-100000000000000005366162204393472.0,-100000000000000005366162204393472.0,-100000000000000005366162204393472.0,-100000000000000005366162204393472.0,CASSINI ORBITER SATURN ISSNA/ISSWA 2 EDR VERSI...,CO,EDR,ISS_EDR
1,N1454726579_1.IMG,data/1454725799_1455008789/N1454726579_1.IMG,COISS_2001,OFF,14.860078,N/A,OPNAV_864_3.ioi ...,8,0.0,12BIT,...,89.647635,TITAN ...,-100000000000000005366162204393472.0,-100000000000000005366162204393472.0,-100000000000000005366162204393472.0,-100000000000000005366162204393472.0,CASSINI ORBITER SATURN ISSNA/ISSWA 2 EDR VERSI...,CO,EDR,ISS_EDR
2,N1454727359_1.IMG,data/1454725799_1455008789/N1454727359_1.IMG,COISS_2001,OFF,14.87867,N/A,OPNAV_880_3.ioi ...,8,0.0,12BIT,...,89.679084,HYPERION ...,-100000000000000005366162204393472.0,-100000000000000005366162204393472.0,-100000000000000005366162204393472.0,-100000000000000005366162204393472.0,CASSINI ORBITER SATURN ISSNA/ISSWA 2 EDR VERSI...,CO,EDR,ISS_EDR
3,N1454728139_1.IMG,data/1454725799_1455008789/N1454728139_1.IMG,COISS_2001,OFF,14.842465,N/A,OPNAV_912_3.ioi ...,8,0.0,12BIT,...,88.677516,PHOEBE ...,-100000000000000005366162204393472.0,-100000000000000005366162204393472.0,-100000000000000005366162204393472.0,-100000000000000005366162204393472.0,CASSINI ORBITER SATURN ISSNA/ISSWA 2 EDR VERSI...,CO,EDR,ISS_EDR
4,N1454728919_1.IMG,data/1454725799_1455008789/N1454728919_1.IMG,COISS_2001,OFF,14.86497,N/A,OPNAV_896_3.ioi ...,8,0.0,12BIT,...,89.79891,IAPETUS ...,-100000000000000005366162204393472.0,-100000000000000005366162204393472.0,-100000000000000005366162204393472.0,-100000000000000005366162204393472.0,CASSINI ORBITER SATURN ISSNA/ISSWA 2 EDR VERSI...,CO,EDR,ISS_EDR
